# This notebook contains the additional analysis part of the paper

- To use any part of this notebook, load the ```model dataset``` and ```validation set```
- Perform the FGSM attack using the validation set and follow the foll. procedure

In [ ]:
# dependencies
import os 
os.environ["GIT_PYTHON_REFRESH"] = "quiet" 
#!module load git
import foolbox as fb
import torch
import eagerpy as ep
from foolbox import PyTorchModel, accuracy, samples
import numpy as np
from n2gem.metrics import gem_build_coverage, gem_build_density
from n2gem.aux_funcs import gem_build_tree
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from util_models import *

In [ ]:
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from fastai.vision.all import *
from fastai.vision import *

In [ ]:
import medmnist
from medmnist import INFO, Evaluator

In [ ]:
from utils import *

Fix the seed generator

In [ ]:
torch.manual_seed(42) 
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
np.random.seed(42)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#print(device); #print(torch.cuda.memory_allocated())
#torch.cuda.device_count()

--------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------

# Run these cells before analysis

Only the example for MNIST dataset is showed here. Follow the same steps as in ```adversarial_attack_``` to perform the same for all the dataset

In [ ]:
# load the respective models for the attack
MyModel = MnNet() # MnNet, PathNet, OrgNet

# mnist
MyModel.load_state_dict(torch.load('chkpt_files/fastai_MnNet_weights.pth', map_location=device))

MyModel.eval()
#print(MyModel)

### Create a Pytorch model for foolbox attack

In [ ]:
# mnist
preprocess = dict(mean=0.1307, std=0.3081)

bound = (0, 1)
original_model = fb.PyTorchModel(MyModel, bounds=bound, preprocessing=preprocess)

- Attack with 20 values of epsilons

In [ ]:
attack2 = fb.attacks.FGSM()
epsilon = np.linspace(0.0, 1, num=20)

#### Load the the images

 mnist

In [ ]:
md_images = torch.load('images/model_dataset_images.pt', map_location='cpu').cpu()
md_labels = torch.load('images/model_dataset_labels.pt', map_location='cpu').cpu()

vali_images = torch.load('images/validation_images.pt', map_location='cpu')
vali_labels = torch.load('images/validation_labels.pt', map_location='cpu')

In [ ]:
# split the model_dataset to obtain 20000 images for the attack

# mnist
_, X_images, _, y_labels = train_test_split(md_images.numpy(), md_labels.numpy(), test_size=0.29455, random_state=42, stratify=md_labels.numpy())


Reshape and form the model_dataset tensors--> named as images

In [ ]:
# mnist
images = ep.astensor(torch.from_numpy(np.array(X_images)).to(device))

images.shape

In [ ]:
# images from the validation set
# resize the real images
real = images.raw.view(images.shape[0], -1)
#real.shape, type(real)

gen_validate = torch.from_numpy(np.array(vali_images).reshape(len(vali_images), -1)).to(device)
gen_labels = torch.from_numpy(np.array(vali_labels).reshape(-1)).to(device)
print(gen_validate.shape)


density_validate = gem_build_density(real, real.shape[0], gen_validate, 'indexflatl2')
coverage_validate = gem_build_coverage(real, real.shape[0], gen_validate, 'indexflatl2')
print(f'density: {density_validate:.5f}, coverage: {coverage_validate:.5f}')

#### Attack the model using validation dataset

- convert the validation_images/labels into ep.tensor for foolbox attack compatibility

In [ ]:
vali_imagesx = ep.astensor(torch.from_numpy(np.array(vali_images)).to(device))
vali_labelsx = ep.astensor(torch.from_numpy(np.array(vali_labels).reshape(-1)).to(device))
print(vali_imagesx.shape, vali_labelsx.shape)

### Perform the attack

In [ ]:
#from utils import model_attack
adv_vali, adv_info_vali = model_attack(attack2, original_model, vali_imagesx, vali_labelsx, epsilon)

## Pymde visualization

In [ ]:
# the reduced dimensionality plots of the adv. samples

pred = []
emb = []
epi = [0,0.25,0.5,0.75,1] # specific epsilon values
t = [0,5,10,15,-1] # specfic adv. samples 
for i in tqdm(t): 
    
    with torch.no_grad():
        image = adv_vali[i].raw.cpu().to(device)
        predict = MyModel(image)
        predictions = np.argmax(predict.cpu().numpy(), axis=1)
    
    # single epsilon adv images
    img = adv_vali[i].raw.view(adv_vali[i].shape[0], -1)
    
    embdd = pymde.preserve_neighbors(img, embedding_dim=2, constraint=pymde.Standardized()).embed() 
    
    pred.append(predictions)
    emb.append(embdd)

The pymde plots (code has been adapted from this part of the pymde repo- plot section(https://github.com/cvxgrp/pymde/blob/main/pymde/problem.py))

In [ ]:
fig, ax = plt.subplots(1,5, figsize=(14,7))
fig.subplots_adjust(hspace=0.05, wspace=0.05)
c = 0
ticks = np.arange(0,10,1).tolist()
a = [16,17,18,19,20]
episo = np.arange(0, 1.05, 0.05)
episo = np.delete(episo, 18)
xtick = [-3,0,3]
k = 0
images = []
for i in range(5):
    im = ax[i].scatter(emb[k][:,0], emb[k][:,1], c=pred[k], s=1.0, cmap="Spectral")

    ax[i].label_outer()
    #ax[i].set_aspect('equal', adjustable='datalim')
    #ax[i,j].set_aspect("equal", adjustable="box")
    ax[i].set_facecolor('gray')
    ax[i].set_title((r'$\epsilon$' + f"={epi[k]:.2f}"))
   

    lim_low = min(np.min(emb[k][:,0].numpy()), np.min(emb[k][:,1].numpy())) * 1.1
    lim_high = max(np.max(emb[k][:,0].numpy()), np.max(emb[k][:,1].numpy())) * 1.1
    lim = _square_lim(lim_low, lim_high)
    ax[i].set_xlim(lim)
    ax[i].set_ylim(lim)
    ax[i].set_aspect("equal", adjustable="box")
    k+= 1
#aspect = 30
#divider = make_axes_locatable(im)
#cax = divider.append_axes("right", size="3.%", pad=0.05)

cbar = fig.colorbar(im, ax=ax, location='bottom', boundaries=np.arange(11) - 0.5, pad=0.1, fraction=.1, shrink=0.5) #orientation='vertical'
cbar.set_ticks(np.arange(11).tolist())
#cbar.ax.set_yticklabels(ticks)
plt.savefig('mnist_pymde.png', bbox_inches = 'tight')

## The realtive change in cluster volumes

In [ ]:
# build a faiss tree on the validation images and query the adv. samples

real_vali = vali_imagesx.raw.view(vali_imagesx.shape[0], -1)
real_vali.shape
vali_tree = gem_build_tree(real_vali, real_vali.shape[0])

In [ ]:
#vali_tree = gem_build_tree(real_vali, real_vali.shape[0])
cluster = []; value =[]

for j in range(len(epsilon)):
    cluster = []
    with torch.no_grad():
        image = adv_vali[j]
        predict = MyModel(image)
        predictions = np.argmax(predict.cpu().numpy(), axis=1)
    for i in range(9):
        idx = np.where(predictions==i)[0]
        if(len(idx) == 0):
            real_radii = 0
            cluster.append(real_radii)
        else:
            query_imgs = adv_vali[j][idx].reshape(idx.shape[0], -1)
            D, I = vali_tree.search(query_imgs, 25)
            #print(la.matrix_norm(D))
            #real_radii = torch.max(torch.sqrt(D), dim=1)[0].mean()
            #real_radii = (torch.sqrt(D).mean(dim=1)).mean()
            real_radii = ((torch.sqrt(D).max(dim=1))[0]**3 * np.pi * (4.0/3.0)).sum()
            cluster.append(real_radii.cpu())
    value.append(cluster)
#cluster = np.array(cluster)

In [ ]:
# compute the relative change in volume between each pred. class clusters
clust = []
for i in range(len(value)):
    a = value[i].reshape(value[i].shape[0], -1)
    with np.errstate(divide='ignore', invalid='ignore'):
        s = abs(a - a[:,None])
        idx = np.isnan(s)
        idxx = np.isinf(s)
        s[idx] = 0
        s[idxx] = 0
        mat = np.linalg.norm(s, axis=-1)
    clust.append(s)

In [ ]:
from matplotlib import colors
fig, ax = plt.subplots(1,5, sharex=True, sharey=True, figsize=(10,10))
fig.subplots_adjust(hspace=0.05, wspace=0.05)
c = 0
ticks = np.arange(0,9,1).tolist()
a = [16,17,18,19,20]
epi = [0,0.25,0.5,0.75,1]
episo = np.arange(0, 1.05, 0.05)
episo = np.delete(episo, 18)
f = [0,5,10,15,19]

images = []
for i,j in zip(range(5),f):
    
    images.append(ax[i].imshow(clust[j], cmap=plt.cm.magma))
    ax[i].label_outer()
    ax[i].set_aspect('equal')
    
    ax[i].set_xticks(ticks)
    ax[i].set_yticks(ticks) 
    ax[i].set_title((r'$\epsilon$' + f"={epi[i]:.2f}"))
#vmin = min(image.get_array().min() for image in images)
#vmax = max(image.get_array().max() for image in images)
#norm = colors.Normalize(vmin=vmin, vmax=vmax)
#for im in images:
#    im.set_norm(norm)

fig.colorbar(images[0], ax=ax,location='bottom', pad=0.05, fraction=.1, shrink=0.5) #orientation='vertical'

#cbar = fig.colorbar(images[0], ax=ax, location='bottom', boundaries=np.arange(11) - 0.5, pad=0.1, fraction=.1, shrink=0.5) #orientation='vertical'
#cbar.set_ticks(np.arange(11).tolist())

#for im in images:
#    im.callbacks.connect('changed', update)
#fig.tight_layout()
#plt.show()
#plt.savefig('mnist_vol_norm.png',bbox_inches='tight')

# Time analysis between ```prdc``` & ```n2gem```

In [ ]:
real_features = []
fake_features = []
samples = [1000, 5000, 10000, 25000, 50000]
for i in tqdm(samples):
    real_features.append(np.random.normal(loc=0.0, scale=1.0,
                                 size=[i, feature_dim]))
    fake_features.append(np.random.normal(loc=0.0, scale=1.0,
                                 size=[i, feature_dim]))

In [ ]:
for i in range(len(samples)):
    s_time = time.time()
    metrics = compute_prdc(real_features=real_features[i],
                       fake_features=fake_features[i],
                       nearest_k=nearest_k)
    e_time = time.time()
    print("time: ", e_time - s_time)

In [ ]:
for i in range(len(samples)):
    s_time = time.time()
    den = gem_build_density(real_features[i], real_features[i].shape[0], fake_features[i], 'indexflatl2')
    cov = gem_build_coverage(real_features[i], real_features[i].shape[0], fake_features[i], 'indexflatl2')
    e_time = time.time()
    print("time: ", e_time - s_time)


In [ ]:
import matplotlib.pyplot as plt
pr = <endter the prdc time value>     #[0.777, 2.116, 6.903, 35.56, 150.285]
n2cpu = <enter the cpu time value of n2gem prdc> # [0.29, 0.633, 0.752, 4.11, 13.33]
n2gpu = <enter the gpu time value of n2gem prdc> # [0.22, 0.33, 0.38, 0.66,1.465]

fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].plot(samples, pr, c='b', ls='-.', marker='o', label='prdc')
ax[0].set_xlabel("sample size")
ax[0].set_ylabel("time(s)")
ax[0].set_title("prdc", fontweight="bold")
ax[0].legend()
ax[1].plot(samples, n2cpu, c='g', ls='-.', marker='+',label='cpu_run')
ax[1].plot(samples, n2gpu, c='r', ls='--', marker='+',label='gpu_run')
ax[1].set_xlabel("sample size")
ax[1].set_ylabel("time(s)")
ax[1].set_title("Our implementation", fontweight="bold")
ax[1].legend()
plt.savefig('time_analysis.png', bbox_inches='tight')

--------------------------------------------------------------------------------------------------------------------------------

### Metrics for FGSM Attack - stats

- to use the following cells, the dataset needs to be loaded and the attack has to be performed


- for the computation of confidence bands
- ```Batch_modelvali_metrics``` - density & coverage between model_dataset& validation_dataset in batches of images
- ```Batch_modelvali_adv_metrics``` - attack the model in batches using validation_dataset. Computed density & coverage between model_dataset & validation_adv samples

 #### Load the images directly from Attack the model section

In [ ]:
# mnist
#realx = torch.from_numpy(np.array(X_images)).view(len(X_images), -1).to(device)

# pathmnist
#realx = md_images.reshape(md_images.shape[0], -1).to(device)
vali_attk_images = torch.from_numpy(np.array(vali_images)).to(device)
vali_attk_labels = torch.from_numpy(np.array(vali_labels).reshape(-1)).to(device)
epsilon = np.linspace(0.0, 1, num=20)


In [ ]:
realx.shape, vali_attk_images.shape, vali_attk_labels.shape

In [ ]:
Batch_modelvali_metrics = model_validation_metrics_batches(realx, vali_attk_images, vali_attk_labels)

In [ ]:
Batch_modelvali_adv_metrics = batch_validation_attack(real, vali_attk_images, vali_attk_labels, epsilon)

In [ ]:
with open(<f_name>, 'w') as newfile:
    newfile.write("# FGSM attack with model_dataset(30547 images)" + "\n" +
                 "# and validation_images(5359) 20 epsilon in batches of 100 images" + "\n" +
                 "# metrics between model_dataset & validation dataset" + "\n" +
                 "# Model_vali_density Model_vali_coverage" + "\n" )
with open('FGSM_attack_pathmnist_1C_100batch_metrics.dat', 'a') as newfile:
    np.savetxt(newfile, np.array(Batch_modelvali_metrics))

In [ ]:
with open(<f_name>, 'w') as newfile:
    newfile.write("# FGSM attack with model_dataset(30547 images)" + "\n" +
                 "# and validation_images(5359) 20 epsilon in batches of 100 images" + "\n" +
                 "# metrics between model_dataset & adv validation dataset" + "\n" +
                 "# Model_vali_adv_density Model_vali_adv_coverage" + "\n" )
with open('FGSM_attack_pathmnist_1C_100batch_adv_metrics.dat', 'a') as newfile:
    np.savetxt(newfile, np.array(Batch_modelvali_adv_metrics))

--------------------------------------------------------------------------------------------------------------------------------

### Metrics for Boundary Attack - stats
### Attack the model with validation_dataset in batches

- create the adversarial samples for each batch exclusively
- use these images as starting_points for boundary attack

#### Get the adversarial starting points for each Batch
- load the 42Batch_Vali_Adv_LSBU.pt file for the saved LSBU adversarials( batch size of 42 images)

In [ ]:
batches = np.arange(0, <vali_sample_size>, 100)
success = []
batch_adv_samples = []
for i in range(len(batches)-1):
    start = batches[i]; end= batches[i+1]
    _, batch_adv_lsbu, adv_bdy_info = n_attack(original_model, vali_attk_images[start:end,...], vali_attk_labels[start:end], epsilons=None)
    print(f"acc: {1 - adv_bdy_info.float32().mean(axis=-1)}")
    success.append(adv_bdy_info); batch_adv_samples.append(batch_adv_lsbu.raw)

In [ ]:
batch_adv_lsbu_samples = torch.stack(batch_adv_samples)
torch.save(batch_adv_lsbu_samples, '140Batch_Vali_Adv_LSBU.pt')

In [ ]:
batch_lsbu_adv = torch.load('140Batch_Vali_Adv_LSBU.pt', map_location='cpu').to(device)

### BoundaryAttack on the model using batches of validation_dataset

- form batches of validation_images
- perform attck using these images
- various batch_sizes:50, 100, 150, 200, 250, 300 (dependent on the dataset)

In [ ]:
# 53, 100, 150, 200, 250, 300 
batches = np.arange(0, 4707, 49)
batch_bdy_adv = []
batch_bdy_info = []
for i in range(len(batches)-1):
    start = batches[i]; end= batches[i+1]
    _, adv_bdy, adv_bdy_info = BdyAttack(original_model, vali_attk_images[start:end,...], vali_attk_labels[start:end], 
                                          epsilons=None) #starting_points=batch_lsbu_adv[i],
    batch_bdy_adv.append(adv_bdy); batch_bdy_info.append(adv_bdy_info)
    if(i%10==0):print(f"batch{i}")

### Metrics

- #### model_dataset & validation_dataset in batches
- #### model_deataset & validation adv in batches

In [ ]:
batch_density_validate = []
batch_coverage_validate = []
batch_model_density = []
batch_model_coverage = []
real = real.to(device)

for i in range(len(batches)-1):
    
    # reference metric
    # model_dataset & validation_dataset in batches
    start = batches[i]; end= batches[i+1]
    bth_val_imgs = vali_attk_images[start:end,...]
    gen_validatex = bth_val_imgs.raw.view(bth_val_imgs.shape[0], -1)
    gen_labelsx = vali_attk_labels[start:end].raw
    density_validatex = gem_build_density(real, real.shape[0], gen_validatex, 'indexflatl2')
    coverage_validatex = gem_build_coverage(real, real.shape[0], gen_validatex, 'indexflatl2')
    
    # adversarial metric
    # model_dataset & validation_dataset adversarials in batches
    gen_adv_valx = batch_bdy_adv[i].raw.view(batch_bdy_adv[i].shape[0], -1)
    model_density_valx = gem_build_density(real, real.shape[0], gen_adv_valx, 'indexflatl2')
    model_coverage_valx = gem_build_coverage(real, real.shape[0], gen_adv_valx, 'indexflatl2')
    
    batch_density_validate.append(density_validatex)
    batch_coverage_validate.append(coverage_validatex)
    batch_model_density.append(model_density_valx)
    batch_model_coverage.append(model_coverage_valx)


### Save metrics to file

- info. in each file
    - attack name with dataset info.
    - density metric btw. model_dataset & validation_Set
    - density metric btw. model_dataset & adv. samples
    - coverage metric btw. model_dataset & validation_Set
    - coverage metric btw. model_dataset & adv. samples
    - Columns: Batch_number | model_validation_densiy | model_adv_density | model_validation_coverage
        model_adv_coverage

In [ ]:
with open('Boundary_attack_organmnist_100batch_metrics.dat', 'w') as newfile:
    newfile.write("# Boundary attack with validation_dataset(4708 images)" + "\n" +
                 "# and metrics using model_dataset(54142)" + "\n" +
                 "# Model_dataset & validation set: density: " + str(density_validate.cpu().numpy()) + "\n" +
                 "# Model_dataset & validation adv (4708): density: " + str(model_density_val.cpu().numpy()) + "\n" +
                 "# Model_dataset & validation set: coverage: " + str(coverage_validate.cpu().numpy()) + "\n" +
                 "# Model_dataset & validation adv (4708): coverage: " + str(model_coverage_val.cpu().numpy()) + "\n" + 
                 "# batch_no model_val_density model_val_adv_density model_val_coverage model_val_adv_coverage" + "\n")
    
with open('Boundary_attack_organmnist_100batch_metrics.dat', 'a') as nefile:
    for i in range (len(batch_density_validate)):
        nefile.write(str(i)+ " " + str(batch_density_validate[i].cpu().numpy())+ " " + str(batch_model_density[i].cpu().numpy())+ " " +
                    str(batch_coverage_validate[i].cpu().numpy()) + " " + str(batch_model_coverage[i].cpu().numpy()) + "\n")

-------------------------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------------------------

The following results are not included in the paper and can be avoided

## PYMDE visualizations of feature extraction

- The features from the intermediate layers were extracted and were visualized using pymde
- These results are not included in the paper and can be omitted

In [ ]:
# pymde plot for model_dataset images( both model and FX_model)
dimPlot(real_FX, y_labels, None)
plt.tight_layout()
plt.savefig('pathmnist_pymde/FXmodel.png')

In [ ]:
dimPlot(real_FX, np.array(y_labels),None)
plt.tight_layout()
plt.savefig('mnist_pymde/FXmodel_20000.png')

In [ ]:
adv_predictions = []
for i in range(len(epsilon)):
    with torch.no_grad():
        image = adv_vali[i].raw.cpu()
        predict = MyModel(image)
        predictions = np.argmax(predict.cpu().numpy(), axis=1)
    #adv_predictions.append(predictions)
    dimPlot(adv_vali[i].raw.reshape(adv_vali[i].shape[0], -1), predictions,None)# ('orgmnist_adv_epi_'+str(i)+'.png'))
    plt.title(f'epi={(epsilon[i]):.3f}', loc='right')
    plt.tight_layout()
    plt.savefig('pathmnist_pymde/'+str(i)+'.png')

#### Pymde for adv. samples

- predict the labels for generated adv. samples
- extract features for each epsilon of adv. samples
- pymde plot the featured extracted adv. samples with predicted labels

In [ ]:
for i in range(len(epsilon)): 
    
    with torch.no_grad():
        image = adv_vali[i].raw.cpu().to(device)
        predict = MyModel(image)
        predictions = np.argmax(predict.cpu().numpy(), axis=1)
    
    # single epsilon adv images
    adv_img = adv_vali[i].raw
    
    # get the feature vector from adv. samples
    gen = torch.from_numpy(feature_extractor(adv_img, FXMyModel, None))
    
    dimPlot(gen, predictions, None)# ('orgmnist_adv_epi_'+str(i)+'.png'))
    plt.title(f'epi={(epsilon[i]):.3f}', loc='right')
    plt.tight_layout()
    plt.savefig('pathmnist_pymde/FXadv_pred'+str(i)+'.png')